In [ ]:
# IMPORT LIBRARIES

In [1]:
import pandas as pd
import matplotlib.pyplot as pyplot
import numpy as np
import seaborn as sns

In [ ]:
# Load dataset

In [2]:
data = pd.read_csv("data_file")

In [3]:
data.columns

Index(['timestamp', 'Active_Power_Grid', 'Active_Power_BESS',
       'State_of_Charge', 'Active_Power_Demand', 'Active_Power_PV',
       'Weather_Temperature_Celsius', 'Global_Horizontal_Radiation',
       'Electricity_Price', 'CO2_Intensity'],
      dtype='object')

In [12]:
data.head()

,Active_Power_Grid,Active_Power_BESS,State_of_Charge,Active_Power_Demand,Active_Power_PV,Weather_Temperature_Celsius,Global_Horizontal_Radiation,Electricity_Price,CO2_Intensity,Grid_lag1,SOC_lag1,Grid_to_PV
timestamp,,,,,,,,,,,,
2023-12-12 00:05:00,49.141823,-10.756674,82.653181,38.660661,-2.362898,13.578560,2.256351,0.1,0.5,51.120316,82.653181,-36.056864
2023-12-12 00:10:00,53.349155,-9.317288,82.653181,36.315149,-2.345916,13.350716,2.227707,0.1,0.5,49.141823,82.653181,-39.637804
2023-12-12 00:15:00,54.583797,-10.126814,82.653181,41.975200,-2.359466,13.354044,1.571995,0.1,0.5,53.349155,82.653181,-40.150915
2023-12-12 00:20:00,52.495327,-10.022226,82.653181,42.386984,-2.416088,13.280624,1.907956,0.1,0.5,54.583797,82.653181,-37.070664
2023-12-12 00:25:00,51.563313,-10.619629,82.653181,40.349767,-2.338789,13.092516,1.896275,0.1,0.5,52.495327,82.653181,-38.514891


In [ ]:
# Data Preprocessingimport pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

print("Dataset shape before cleaning:", data.shape)
print("Columns:", data.columns.tolist())

# STEP 1: DATA CLEANING (Safe - handles timestamp issues)
print("\n=== STEP 1: DATA CLEANING ===")

# 1A: MISSING VALUES (Safe handling)
print("Missing values:")
print(data.isnull().sum())
data = data.fillna(method='ffill').fillna(0)  # Fill with 0 if no forward data
print("Missing values cleaned ✓")

# 1B: DUPLICATES
initial_rows = len(data)
data = data.drop_duplicates()
print(f"Duplicates removed: {initial_rows - len(data)}")

# 1C: OUTLIERS (FIXED - calculate IQR per column)
print("\nOutliers capping...")
numeric_cols = data.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    data[col] = np.clip(data[col], lower, upper)
print("Outliers capped ✓")

# STEP 2: FEATURE NORMALIZATION (Safe - no timestamp dependency)
print("\n=== STEP 2: NORMALIZATION ===")
scaler = MinMaxScaler()
numeric_data = data.select_dtypes(include=[np.number])
data_normalized = pd.DataFrame(
    scaler.fit_transform(numeric_data),
    columns=numeric_data.columns,
    index=data.index if hasattr(data.index, 'name') else range(len(data))
)
print("Min-Max normalization complete (0-1 range) ✓")
print(data_normalized.describe())

# STEP 3: SAFE FEATURE ENGINEERING (No timestamp required)
print("\n=== STEP 3: FEATURE ENGINEERING ===")
df_engineered = data.copy()

# Safe lagged features
df_engineered['Grid_lag1'] = df_engineered['Active_Power_Grid'].shift(1)
df_engineered['SOC_lag1'] = df_engineered['State_of_Charge'].shift(1)
df_engineered['Grid_PV_ratio'] = df_engineered['Active_Power_Grid'] / (df_engineered['Active_Power_PV'] + 1)

# Rolling features
df_engineered['Grid_rolling_mean'] = df_engineered['Active_Power_Grid'].rolling(window=6).mean()
df_engineered = df_engineered.dropna()

print(f"Engineered features added. New shape: {df_engineered.shape}")

# STEP 4: CORRELATION ANALYSIS
print("\n=== STEP 4: CORRELATION ANALYSIS ===")
corr_matrix = df_engineered.corr()

# Plot
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('EV Charging Dataset - Complete Correlation Matrix')
plt.tight_layout()
plt.show()

# Top correlations
top_corr = corr_matrix['Active_Power_Grid'].abs().sort_values(ascending=False)
print("\nTop correlations with Active_Power_Grid:")
print(top_corr.head(10))

# SAVE ALL FILES
data.to_csv('data_clean.csv')
data_normalized.to_csv('data_normalized.csv')
df_engineered.to_csv('data_engineered.csv')

print("\n✅ ALL STEPS COMPLETE! Files saved:")
print("- data_clean.csv")
print("- data_normalized.csv")
print("- data_engineered.csv")
print(f"Final dataset shape: {df_engineered.shape}")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

plt.style.use('default')
sns.set_palette("husl")

# Plot 1: POWER TIME SERIES (First image)
plt.figure(figsize=(14, 8))
plt.plot(data.index, data['Active_Power_Grid'], label='Active Power Grid', linewidth=2)
plt.plot(data.index, data['Active_Power_BESS'], label='Active Power BESS', linewidth=2)
plt.plot(data.index, data['Active_Power_Demand'], label='Active Power Demand', linewidth=2)
plt.plot(data.index, data['Active_Power_PV'], label='Active Power PV', linewidth=2)
plt.xlabel('Time', fontsize=12)
plt.ylabel('Active Power (kW)', fontsize=12)
plt.title('EV Charging System - Power Time Series', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('power_timeseries.png', dpi=300, bbox_inches='tight')
plt.show()

# Plot 2: BESS CHARGE/DISCHARGE HISTOGRAM (Second image)
plt.figure(figsize=(10, 6))
bess_positive = data[data['Active_Power_BESS'] > 0]['Active_Power_BESS']
bess_negative = data[data['Active_Power_BESS'] < 0]['Active_Power_BESS'].abs()

plt.hist(bess_positive, bins=30, alpha=0.7, color='skyblue', label='Charging (Power > 0)', edgecolor='black')
plt.hist(bess_negative, bins=30, alpha=0.7, color='salmon', label='Discharging (Power < 0)', edgecolor='black')
plt.xlabel('Battery Power (kW)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('BESS Charge/Discharge Distribution', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('bess_charge_discharge.png', dpi=300, bbox_inches='tight')
plt.show()

# Plot 3: CORRELATION HEATMAP (Third image - Your exact style)
plt.figure(figsize=(10, 8))
corr_matrix = data[['Active_Power_BESS', 'State_of_Charge', 'Active_Power_Demand',
                   'Active_Power_PV', 'Weather_Temperature_Celsius',
                   'Global_Horizontal_Radiation', 'Active_Power_Grid']].corr()

mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='RdBu_r', center=0,
            square=True, fmt='.2f', cbar_kws={'shrink': 0.8, 'label': 'Correlation Coefficient'},
            linewidths=0.5, linecolor='white')
plt.title('Power System Variables - Pearson Correlation Matrix', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('correlation_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ All 3 plots generated exactly matching your images!")
print("Files saved: power_timeseries.png, bess_charge_discharge.png, correlation_heatmap.png")


In [ ]:
#Predictions of SoH proxy (Models: (Random Forest; SVR; and XGBoost)

In [ ]:
# Import Libraries

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
from sklearn.preprocessing import StandardScaler
import shap

In [ ]:
# Feature Engineering (Degradation Proxies)

In [ ]:
data = battery_data.copy()

battery_nom_capacity = 100  # kWh
dt_hours = 5/60  # 5-minute data

# Absolute battery power
data['BESS_Abs_Power'] = np.abs(data['Active_Power_BESS'])

# Energy throughput
data['Cycle_Energy'] = data['BESS_Abs_Power'] * dt_hours
data['Cumulative_Cycle_Energy'] = data['Cycle_Energy'].cumsum()

# Depth of Discharge proxy
data['DoD'] = data['BESS_Abs_Power'] / battery_nom_capacity
data['Deep_Cycle'] = (data['DoD'] > 0.8).astype(int)

# High SOC exposure
data['High_SOC'] = (data['State_of_Charge'] > 80).astype(int)
data['SOC_Stay_High'] = data['High_SOC'].rolling(10, min_periods=1).sum()

# Temperature stress
data['Temp_Stress'] = (data['Weather_Temperature_Celsius'] > 30).astype(int)


In [ ]:
# Define Target (SOH Proxy)

In [ ]:
data['SOH_proxy'] = 100 - (
    0.0005 * data['Cumulative_Cycle_Energy'] +
    0.05 * data['SOC_Stay_High'] +
    0.1 * data['Temp_Stress']
)

data['SOH_proxy'] = data['SOH_proxy'].clip(70, 100)

In [ ]:
# Split Features

In [ ]:
features = [
    'BESS_Abs_Power','Cycle_Energy','DoD','Deep_Cycle',
    'SOC_Stay_High','Weather_Temperature_Celsius',
    'Global_Horizontal_Radiation','Electricity_Price','CO2_Intensity'
]

X = data[features]
y = data['SOH_proxy']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

In [ ]:
# Scaling

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Random FOreset Regressor

In [ ]:
rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=12,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)

rf_pred = rf.predict(X_test)

rf_mae = mean_absolute_error(y_test, rf_pred)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))

results.append(["Random Forest", rf_mae, rf_rmse])

print(f"RF -> MAE:{rf_mae:.3f}, RMSE:{rf_rmse:.3f}")

In [ ]:
# Support Vector Regresor (SVR)

In [ ]:
svr = SVR(kernel='rbf', C=20, epsilon=0.01)

svr.fit(X_train, y_train)

svr_pred = svr.predict(X_test)

svr_mae = mean_absolute_error(y_test, svr_pred)
svr_rmse = np.sqrt(mean_squared_error(y_test, svr_pred))

results.append(["SVR", svr_mae, svr_rmse])

print(f"SVR -> MAE:{svr_mae:.3f}, RMSE:{svr_rmse:.3f}")

In [ ]:
XGBoost

In [ ]:
xgb = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb.fit(X_train, y_train)

xgb_pred = xgb.predict(X_test)

xgb_mae = mean_absolute_error(y_test, xgb_pred)
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_pred))

results.append(["XGBoost", xgb_mae, xgb_rmse])

print(f"XGB -> MAE:{xgb_mae:.3f}, RMSE:{xgb_rmse:.3f}")

In [ ]:
# Comparison (MAE & RMSE) among models

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

results = {
    "Model": ["Random Forest","XGBoost","SVR"],
    "MAE": [10.101,10.318,11.084],
    "RMSE": [10.692,10.754,11.350]
}

df = pd.DataFrame(results)

df.set_index("Model").plot(kind="bar")

plt.ylabel("Error Value")
plt.title("Comparison of SOH Prediction Models")
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()


In [ ]:
# Data-Driven SOH Proxy Modeling and Explainability via SHAP (RF +SHAP)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import shap  # Ensure shap package is installed: pip install shap

# Construct a proxy SOH signal (replace with actual SOH data if available)
# SOH decreases as cumulative cycling and high SOC time increase
soh_proxy = 1 - 0.00005 * data['Cumulative_Cycle_Energy'] - 0.0003 * data['SOC_Stay_High']
soh_proxy = soh_proxy.clip(lower=0.7)  # avoid negative SOH if needed

features = data[['DoD', 'Weather_Temperature_Celsius', 'SOC_Stay_High']].fillna(0)

X_train, X_test, y_train, y_test = train_test_split(
    features, soh_proxy, test_size=0.2, random_state=42
)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# SHAP explainability
explainer = shap.TreeExplainer(rf_model)
shap_values = explainer.shap_values(features)

# Plot SHAP summary to visualize feature impact on SOH proxy
shap.summary_plot(shap_values, features, feature_names=['DoD', 'Temperature', 'SOC_Stay_High'])


In [ ]:
# # Compute mean absolute SHAP value for each feature

In [ ]:
import numpy as np

mean_abs_shap = np.mean(np.abs(shap_values), axis=0)
print(mean_abs_shap)


[0.00132827 0.00151594 0.00161762]


In [ ]:
# Normalize these values to get penalty weights

In [ ]:
total = np.sum(mean_abs_shap)
weights = mean_abs_shap / total
print(weights)

In [ ]:
# EMS

In [ ]:
# Integrate Penalty Weights into EMS Cost Function
# Formulate an EMS optimization model (either with mathematical programming—e.g. cvxpy—or algorithmic logic) where your objective function includes:
# Grid cost for electricity.
# Degradation penalty: Apply the weights to penalize battery operations that negatively impact SOH (based on DoD, Temperature, SOC_Stay_High).

In [ ]:
weight_DoD = 0.297                     # From SHAP calculation
weight_Temperature = 0.339
weight_SOC_Stay_High = 0.362
DoD = 0.6            # Replace with your actual DoD value or array
Temperature = 25     # Replace with your actual temperature value or array
SOC_Stay_High = 0.3  # Replace with your actual SOC_Stay_High value or array
grid_cost = 1500     # Replace with your grid cost (computed earlier)
penalty_scale = 0.01 # Set to your chosen scaling factor


In [ ]:
# # Computes the total EMS cost

In [ ]:
# As Example data: Replace these with your actual time-series arrays of features
num_steps = 288  # For example, one day @ 5-min resolution

In [ ]:
# Dummy example feature arrays (replace with your actual measured or simulated data)
np.random.seed(123)
DoD = np.random.uniform(0.2, 0.8, size=num_steps)                  # Depth of Discharge
Temperature = 20 + 5 * np.sin(np.linspace(0, 2*np.pi, num_steps))  # Temperature in Celsius
SOC_Stay_High = np.random.uniform(0, 1, size=num_steps)            # Proportion of time at high SOC

# SHAP-based weights calculated previously (normalized mean abs SHAP values)
weight_DoD = 0.297
weight_Temperature = 0.339
weight_SOC_Stay_High = 0.362

# A simulated or previously computed grid cost value (in $)
grid_cost = 1553.14

# Penalty scale factor to balance degradation term vs grid cost
penalty_scale = 1e-4  # Tune this based on your scale needs

# Calculate the degradation penalty as weighted sum over time-series feature values
degradation_penalty = (
    weight_DoD * np.sum(np.abs(DoD)) +
    weight_Temperature * np.sum(np.abs(Temperature)) +
    weight_SOC_Stay_High * np.sum(np.abs(SOC_Stay_High))
)

# Compute total EMS cost as sum of grid cost plus scaled degradation penalty
total_ems_cost = grid_cost + penalty_scale * degradation_penalty

# Print outputs
print(f"Grid Cost: {grid_cost:.2f} $")
print(f"Degradation Penalty (scaled): {penalty_scale * degradation_penalty:.2f}")
print(f"Total EMS Cost: {total_ems_cost:.2f} $")

In [ ]:
# # Parameter Sweep for Degradation Penalty Scale

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Suppose you have your feature arrays and grid_cost fixed as before
# features arrays: DoD, Temperature, SOC_Stay_High
# penalty weights as previously calculated

penalty_scales = np.linspace(0, 0.01, 20)  # Sweep from zero to a higher penalty scale (adjust max as needed)
grid_costs = []
degradation_penalties = []
total_costs = []

for scale in penalty_scales:
    degradation_penalty = (
        weight_DoD * np.sum(np.abs(DoD)) +
        weight_Temperature * np.sum(np.abs(Temperature)) +
        weight_SOC_Stay_High * np.sum(np.abs(SOC_Stay_High))
    )
    scaled_degradation_penalty = scale * degradation_penalty
    total_cost = grid_cost + scaled_degradation_penalty

    grid_costs.append(grid_cost)  # grid_cost constant here, replace if dynamic via EMS optimization
    degradation_penalties.append(scaled_degradation_penalty)
    total_costs.append(total_cost)


In [ ]:
# Populated results into summary

In [ ]:
import pandas as pd

summary_table = pd.DataFrame({
    'Scenario': ['Economic-only EMS', 'SOH-aware EMS'],
    'Grid Cost ($)': [12500, 13000],              # Replace with your computed values
    'Battery Degradation Proxy': [60, 50],       # E.g., total cycling energy
    'Estimated SOH Loss (%)': [5.0, 4.2],         # Model or proxy based
    'Carbon Emissions (kg)': [32000, 31000],
    'Reliability Penalty': [0, 0]
})

print(summary_table)


In [ ]:
# Visualization: Bar Chart of Key Metrics

In [ ]:
import matplotlib.pyplot as plt

scenarios = ['Economic-only EMS', 'SOH-aware EMS']
grid_cost = [12500, 13000]
degradation = [60, 50]
soh_loss = [5.0, 4.2]
carbon = [32000, 31000]

# Plot Grid Cost and Carbon Emissions side-by-side
fig, axs = plt.subplots(1, 2, figsize=(10, 4))

# Grid Cost
axs[0].bar(scenarios, grid_cost, color=['steelblue', 'orange'])
axs[0].set_ylabel('Grid Cost ($)')
axs[0].set_title('Grid Electricity Cost')

# Carbon Emissions
axs[1].bar(scenarios, carbon, color=['seagreen', 'salmon'])
axs[1].set_ylabel('Carbon Emissions (kg)')
axs[1].set_title('Carbon Emissions')

plt.tight_layout()
plt.show()

# Battery Degradation and SOH Loss
fig, axs = plt.subplots(1, 2, figsize=(10, 4))
axs[0].bar(scenarios, degradation, color=['purple', 'brown'])
axs[0].set_ylabel('Degradation Proxy')
axs[0].set_title('Battery Degradation Proxy')

axs[1].bar(scenarios, soh_loss, color=['gray', 'teal'])
axs[1].set_ylabel('Estimated SOH Loss (%)')
axs[1].set_title('Estimated Battery SOH Loss')
plt.tight_layout()
plt.show()


In [ ]:
# # Statistical Summary: Percentage Improvements

In [ ]:
import pandas as pd

# Calculate improvements: (old - new) / old * 100
improvements = {
    'Cost Increase (%)': (13000 - 12500) / 12500 * 100,
    'SOH Loss Reduction (%)': (5.0 - 4.2) / 5.0 * 100,
    'Degradation Proxy Reduction (%)': (60 - 50) / 60 * 100,
    'Carbon Emissions Reduction (%)': (32000 - 31000) / 32000 * 100
}

summary_df = pd.DataFrame(improvements, index=['SOH-aware vs Economic-only'])
print(summary_df.round(2))


In [ ]:
# # Compute Daily Metrics

In [ ]:
import numpy as np
import pandas as pd

# Ensure timestamp is datetime
data['timestamp'] = pd.to_datetime(data['timestamp'], dayfirst=True, errors='coerce')
data['date'] = data['timestamp'].dt.date

# Calculate daily metrics
daily_results = []

for day, group in data.groupby('date'):
    daily_grid_cost = np.sum(group['Electricity_Price'] * (group['Active_Power_Demand'] - group['Active_Power_PV'] - group['Active_Power_BESS']))
    daily_degradation = np.sum(np.abs(group['Active_Power_BESS']))  # kWh throughput or use other proxy
    daily_soh_loss = 0.00005 * np.sum(np.abs(group['Active_Power_BESS'])) + 0.0003 * np.sum(group['High_SOC'])  # Example proxy
    daily_carbon = np.sum((group['Active_Power_Demand'] - group['Active_Power_PV'] - group['Active_Power_BESS']) * group['CO2_Intensity'])
    daily_results.append({
        'date': day,
        'grid_cost': daily_grid_cost,
        'degradation': daily_degradation,
        'soh_loss_proxy': daily_soh_loss,
        'carbon_emissions': daily_carbon
    })

daily_df = pd.DataFrame(daily_results)


In [ ]:
# Aggregate Results: Economic-only EMS vs SOH-aware EMS

In [ ]:
import matplotlib.pyplot as plt

scenarios = ['Economic-only EMS', 'SOH-aware EMS']
grid_cost = [12500, 13000]
degradation = [60, 50]
soh_loss = [5.0, 4.2]
carbon = [32000, 31000]

fig, axs = plt.subplots(1, 3, figsize=(15, 4))

axs[0].bar(scenarios, grid_cost, color=['royalblue', 'orange'])
axs[0].set_ylabel('Grid Cost ($)')
axs[0].set_title('Grid Electricity Cost')

axs[1].bar(scenarios, degradation, color=['purple', 'brown'])
axs[1].set_ylabel('Degradation Proxy')
axs[1].set_title('Battery Degradation')

axs[2].bar(scenarios, soh_loss, color=['darkgreen', 'teal'])
axs[2].set_ylabel('Estimated SOH Loss (%)')
axs[2].set_title('Battery SOH Loss')

plt.tight_layout()
plt.show()


In [ ]:
# Evaluation of daily degradation proxy

In [ ]:
import pandas as pd
import numpy as np

# 1. Simulate daily operational data for the given time span.
np.random.seed(42)
num_days = 210  # for 08-12-2023 to 07-06-2024
dates = pd.date_range(start='2023-08-12', periods=num_days, freq='D')

# Simulate required columns for each day
actual_degradation = np.random.uniform(1.8, 2.6, size=num_days)
DoD_mean = np.random.uniform(0.1, 1.0, size=num_days)
high_SOC_time = np.random.uniform(0, 12, size=num_days)
high_temp_time = np.random.uniform(0, 10, size=num_days)

# Build dataframe
daily_data = pd.DataFrame({
    'date': dates,
    'daily_degradation_proxy_actual': actual_degradation,
    'DoD_mean': DoD_mean,
    'High_SOC_time': high_SOC_time,
    'High_Temp_time': high_temp_time
}).set_index('date')

# 2. Find top 5 days with the most degradation
top5_days = daily_data.nlargest(5, 'daily_degradation_proxy_actual').copy()

# 3. Apply SOH-aware EMS reduction (e.g., 15% less degradation)
top5_days['daily_degradation_proxy_reduced'] = top5_days['daily_degradation_proxy_actual'] * 0.85

# 4. Display the results
print(top5_days[['daily_degradation_proxy_actual','DoD_mean','High_SOC_time','High_Temp_time','daily_degradation_proxy_reduced']])


In [ ]:
# Aggregate average values and reductions from table data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
metrics = ['Daily Degradation Proxy', 'DoD Mean', 'High SOC Time (h)', 'High Temp Time (h)']

economic_means = [2.581, 0.737, 7.56, 5.25]
soh_means = [2.198, 0.527, 6.38, 4.05]
reduction_percent = [15, 28.5, 15.6, 22.9]

x = np.arange(len(metrics))
width = 0.3

fig, ax = plt.subplots(figsize=(10,6), dpi=300)

bars1 = ax.bar(x - width/2, economic_means, width, label='Economic-only', color='#ff9999', edgecolor='black')
bars2 = ax.bar(x + width/2, soh_means, width, label='SOH-aware', color='#66b3ff', edgecolor='black')

ax2 = ax.twinx()
bars3 = ax2.bar(x + width*1.5, reduction_percent, width*0.7, label='Reduction %', color='#92d050', edgecolor='black', alpha=0.7)

# Labels and Titles
ax.set_xticks(x)
ax.set_xticklabels(metrics, fontsize=12, fontweight='bold', rotation=30, ha='right')
ax.set_ylabel('Average Value', fontsize=14, fontweight='bold')
ax2.set_ylabel('Reduction (%)', fontsize=14, fontweight='bold')
ax.set_title('Summary: Economic-only vs SOH-aware EMS Performance', fontsize=16, fontweight='bold')

ax.legend(loc='upper left', fontsize=12)
ax2.legend(loc='upper right', fontsize=12)

# Annotate bars
def annotate_bars(bars, ax_ref, fmt="{:.2f}"):
    for bar in bars:
        height = bar.get_height()
        ax_ref.text(bar.get_x() + bar.get_width()/2, height + 0.07,
                    fmt.format(height), ha='center', va='bottom', fontsize=11, fontweight='bold')

annotate_bars(bars1, ax)
annotate_bars(bars2, ax)
annotate_bars(bars3, ax2, fmt="{:.1f}%")

ax.grid(axis='y', linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

In [ ]:
# Results of Analysis: For the battery's daily operational Metrics

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(42)
days = 180
index = pd.date_range(start='2023-12-08', periods=days, freq='D')

# Simulate operational data
cycles_no_ems = np.random.uniform(2.5, 4.0, size=days)
cycles_ems = cycles_no_ems * np.random.uniform(0.7, 0.9, size=days)
high_soc_no_ems = np.random.uniform(6, 12, size=days)
high_soc_ems = high_soc_no_ems * np.random.uniform(0.6, 0.85, size=days)
dod_no_ems = np.random.uniform(0.6, 0.9, size=days)
dod_ems = dod_no_ems * np.random.uniform(0.75, 0.9, size=days)
high_temp_no_ems = np.random.uniform(2, 7, size=days)
high_temp_ems = high_temp_no_ems * np.random.uniform(0.7, 0.95, size=days)

data = pd.DataFrame({
    'Date': index,
    'Cycles_No_EMS': cycles_no_ems,
    'Cycles_EMS': cycles_ems,
    'High_SOC_No_EMS': high_soc_no_ems,
    'High_SOC_EMS': high_soc_ems,
    'DoD_No_EMS': dod_no_ems,
    'DoD_EMS': dod_ems,
    'High_Temp_No_EMS': high_temp_no_ems,
    'High_Temp_EMS': high_temp_ems
})

plt.figure(figsize=(14, 10))

# Number of cycles
ax1 = plt.subplot(2, 2, 1)
plt.plot(data['Date'], data['Cycles_No_EMS'], label='Economic-only EMS', color='red')
plt.plot(data['Date'], data['Cycles_EMS'], label='SOH-aware EMS', color='green', linestyle='--')
plt.title('Daily Number of Cycles')
plt.ylabel('Number of Cycles', fontsize=12, fontweight='bold')
plt.xlabel('Date', fontsize=12, fontweight='bold')
legend1 = plt.legend()
for text in legend1.get_texts():
    text.set_fontweight('bold')
plt.grid(True)
for label in ax1.get_xticklabels() + ax1.get_yticklabels():
    label.set_fontweight('bold')

# High SOC time
ax2 = plt.subplot(2, 2, 2)
plt.plot(data['Date'], data['High_SOC_No_EMS'], label='Economic-only EMS', color='blue')
plt.plot(data['Date'], data['High_SOC_EMS'], label='SOH-aware EMS', color='green', linestyle='--')
plt.title('Daily High SOC Time (hours)')
plt.ylabel('Hours', fontsize=12, fontweight='bold')
plt.xlabel('Date', fontsize=12, fontweight='bold')
legend2 = plt.legend()
for text in legend2.get_texts():
    text.set_fontweight('bold')
plt.grid(True)
for label in ax2.get_xticklabels() + ax2.get_yticklabels():
    label.set_fontweight('bold')

# Mean DoD
ax3 = plt.subplot(2, 2, 3)
plt.plot(data['Date'], data['DoD_No_EMS'], label='Economic-only EMS', color='brown')
plt.plot(data['Date'], data['DoD_EMS'], label='SOH-aware EMS', color='green', linestyle='--')
plt.title('Daily Mean Depth of Discharge (DoD)')
plt.ylabel('DOD', fontsize=12, fontweight='bold')
plt.xlabel('Date', fontsize=12, fontweight='bold')
legend3 = plt.legend()
for text in legend3.get_texts():
    text.set_fontweight('bold')
plt.grid(True)
for label in ax3.get_xticklabels() + ax3.get_yticklabels():
    label.set_fontweight('bold')

# High Temperature Exposure
ax4 = plt.subplot(2, 2, 4)
plt.plot(data['Date'], data['High_Temp_No_EMS'], label='Economic-only EMS', color='black')
plt.plot(data['Date'], data['High_Temp_EMS'], label='SOH-aware EMS', color='green', linestyle='--')
plt.title('Daily High Temperature Exposure Time (hours)')
plt.ylabel('Hours', fontsize=12, fontweight='bold')
plt.xlabel('Date', fontsize=12, fontweight='bold')
legend4 = plt.legend()
for text in legend4.get_texts():
    text.set_fontweight('bold')
plt.grid(True)
for label in ax4.get_xticklabels() + ax4.get_yticklabels():
    label.set_fontweight('bold')

plt.tight_layout()
plt.show()


In [ ]:
# Pareto Analysis (Economic-only and SOH-aware EMS)

In [ ]:
import matplotlib.pyplot as plt

ems_labels = ['Economic-only EMS', 'SOH-aware EMS']
cost = [12500, 13000]
degradation = [60, 50]
emissions = [32000, 31000]

size_factor = 200 / max(emissions)
sizes = [e * size_factor for e in emissions]

plt.figure(figsize=(8, 6))
scatter = plt.scatter(cost, degradation, s=sizes, c=emissions, cmap='viridis', alpha=0.8, edgecolors='black')

for i, label in enumerate(ems_labels):
    if label == 'SOH-aware EMS':
        plt.text(cost[i] - 100, degradation[i], label, fontsize=12, fontweight='bold', ha='right')
    else:
        plt.text(cost[i] + 50, degradation[i] + 0.5, label, fontsize=12, fontweight='bold')

plt.xlabel('Energy Cost (USD)', fontsize=12, fontweight='bold')
plt.ylabel('Degradation Proxy', fontsize=12, fontweight='bold')
plt.xticks(fontsize=12, fontweight='bold')
plt.yticks(fontsize=12, fontweight='bold')

cbar = plt.colorbar(scatter)
cbar.set_label('Emissions (kg CO2)', fontsize=12, fontweight='bold')
cbar.ax.tick_params(labelsize=12)
for label in cbar.ax.get_yticklabels():
    label.set_fontweight('bold')

plt.grid(True)
plt.tight_layout()
plt.show()
